# STAT4012 Project

- Add data augmentation
- Add Dropout layer
- Increase num_epochs to 100
- Increase batch size to 128

In [2]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-7c8786dc-e1bd-3dde-fe50-12894c4f041a)


In [3]:
import multiprocessing as mp
num_cpu = mp.cpu_count()
num_cpu

2

# Training

In [4]:
_exp_name = "alexnet3"

In [5]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import wandb

In [6]:
myseed = 4012  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

## **Transforms**
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [8]:
test_tfm = transforms.Compose([
    # (height = width = 128)
    transforms.Resize((168, 168)),
#     transforms.ToTensor(),
    # Ten Crop in test
    transforms.TenCrop(128), # this is a tuple of PIL Images
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
])

train_tfm = transforms.Compose([
    # (height = width = 99)
    #transforms.CenterCrop(),
    transforms.Resize((168,168)),
#     transforms.RandomResizedCrop((99, 99), scale=(0.75, 1.0)),
    transforms.RandomCrop((128,128) ),
    #transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    # transforms.RandomRotation(180),
    transforms.RandomAffine(30),
    #transforms.RandomInvert(p=0.2),
    #transforms.RandomPosterize(bits=2),
    #transforms.RandomSolarize(threshold=192.0, p=0.2),
    #transforms.RandomEqualize(p=0.2),
    # transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    #transforms.RandomApply(torch.nn.ModuleList([]))
    # PCA augmentation
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [9]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        # print(path)
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label



In [10]:
# alex net
class Classifier(nn.Module):
    def __init__(self, num_classes=11):
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        super().__init__()
        # input [3,99,99]
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 96, 11, 4, 2),     # Conv 1 23*23 *96  
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2, 0),          # MaxPool 11*11*96
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, 1, 2),    # Con 2 11*11*256   
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2, 0),   
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, 1, 1),    # Con 3 11*11*256   
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),   
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, 1, 1),   # Con 4 5*5*384   5
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),   
        )
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(384, 256, 3, 1, 1),   # Con 5 5*5*256   5
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2, 0),          # output 2*2*256
        )
        
#         self.avgpool = nn.AvgPool2d(2)
        
        self.fc = nn.Sequential(            
            nn.Dropout(0.5),
            nn.Linear(256 * 3 * 3, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes),
        )
    
    def forward(self, x):
        # [3, 128, 128]
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = out.flatten(1)
        out = self.fc(out)
        return out

In [11]:
!ls /kaggle/input/food-11-newstruc


food-11


In [12]:
batch_size = 128
_dataset_dir = "/kaggle/input/food-11-newstruc/food-11/"

train_set = FoodDataset(os.path.join(_dataset_dir,"training"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True)
valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True)

One /kaggle/input/food-11-newstruc/food-11/training sample /kaggle/input/food-11-newstruc/food-11/training/0_0.jpg
One /kaggle/input/food-11-newstruc/food-11/validation sample /kaggle/input/food-11-newstruc/food-11/validation/0_0.jpg


In [22]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# The number of training epochs and patience.
n_epochs = 400
patience = 40 # If no improvement in 'patience' epochs, early stop

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5) 

# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0

# wandb.init(
#     project="Food-11",
#     config={
#         "learning rate": 0.0003,
#         "architecture": "CNN",
#         "epochs": n_epochs,
#         "batch_size": batch_size,
#         "image_dim": 128,
#     }
# )

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
#         wandb.log({"step_training_loss": loss.item()})
#         wandb.log({"step_training_accuracy": acc})
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        bs, nc, c, h, w = imgs.size()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            outputs = model((imgs.to(device)).view(-1,c,h,w))
            logits = outputs.view(bs,nc,-1).mean(1)
#             logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
#         wandb.log({"step_validation_loss": loss.item()})
#         wandb.log({"step_validation_accuracy": acc})
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # update logs
    if valid_acc > best_acc:
        with open(f"{_exp_name}_log.txt","a") as f:
            newline = '\n'
            item = f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best{newline}"
            f.write(item)
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"{_exp_name}_log.txt","a") as f:
            newline = '\n'
            item = f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}{newline}"
            f.write(item)
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

#     wandb.log({
#         "average_training_loss": train_loss,
#         "average_validation_loss": valid_loss,
#     })

    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

100%|██████████| 78/78 [01:31<00:00,  1.17s/it]


[ Train | 001/400 ] loss = 2.08798, acc = 0.26569


100%|██████████| 27/27 [00:43<00:00,  1.60s/it]


[ Valid | 001/400 ] loss = 1.95027, acc = 0.30690
[ Valid | 001/400 ] loss = 1.95027, acc = 0.30690 -> best
Best model found at epoch 0, saving model


100%|██████████| 78/78 [01:00<00:00,  1.29it/s]


...

[ Valid | 141/400 ] loss = 1.42154, acc = 0.64376
[ Valid | 141/400 ] loss = 1.42154, acc = 0.64376


100%|██████████| 78/78 [00:59<00:00,  1.30it/s]


[ Train | 142/400 ] loss = 0.29553, acc = 0.90006


100%|██████████| 27/27 [00:35<00:00,  1.33s/it]

[ Valid | 142/400 ] loss = 1.71804, acc = 0.60468
[ Valid | 142/400 ] loss = 1.71804, acc = 0.60468
No improvment 40 consecutive epochs, early stopping


In [17]:
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_cpu, pin_memory=True)

One /kaggle/input/food-11-newstruc/food-11/test sample /kaggle/input/food-11-newstruc/food-11/test/0_0.jpg


# Testing and generate prediction CSV

In [18]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
_dataset_dir = "/kaggle/input/food-11-newstruc/food-11/"

model_best = Classifier().to(device)
model_best.load_state_dict(torch.load(f"/kaggle/input/alex-net-model/{_exp_name}_best.ckpt"))
model_best.eval()
prediction = []
test_accs = []
true_labels = []
with torch.no_grad():
    for data, labels in test_loader:
        bs, nc, c, h, w = data.size()
        outputs = model_best((data.to(device)).view(-1,c,h,w))
        test_pred = outputs.view(bs,nc,-1).mean(1)
#         test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)

        acc = (test_pred.argmax(dim=-1) == labels.to(device)).float().mean()
        test_accs.append(acc)
        prediction += test_label.squeeze().tolist()
        true_labels.extend(labels.tolist())

test_acc = sum(test_accs) / len(test_accs)

print(f"Test accurary: {test_acc}")

Test accurary: 0.7078459858894348


In [25]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("prediction_AlexNet.csv",index = False)

df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = true_labels
df.to_csv("true_labels.csv",index = False)